In [1]:
import pandas as pd
import logging
log = logging.Logger('test')

In [2]:
import os 
os.chdir("..")

In [3]:
%run src/assign-unique-ids.py

2024-04-06T11:55:35[INFO]: running assign-unique-ids.py


/Users/ashwin/opt/anaconda3/envs/cpdp/lib/python3.7/site-packages/IPython/utils/py3compat.py:168: DtypeWarning: Columns (8) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(compiler(f.read(), fname, 'exec'), glob, loc)


KeyError: 'sr_no'

In [2]:
from assign_unique_ids_functions import *
from general_utils import *

In [3]:
df = pd.read_csv("../input/TRR-subjects_2004-2016_2016-09.csv.gz")
#df.insert(0,'test',1)
df.columns

Index(['trr_id', 'gender', 'race', 'age', 'birth_year', 'armed', 'injured',
       'alleged_injury', 'subject_no', 'sr_no', 'se_no', 'trr_date'],
      dtype='object')

In [ ]:
df.loc[keep_duplicates(df[['subject_no', 'sr_no', 'se_no']].dropna(how='any').drop_duplicates(), 'subject_no').index]

In [ ]:
srs = df.loc[keep_duplicates(df[['subject_no', 'sr_no', 'se_no']].dropna(how='any').drop_duplicates(), 'subject_no').index].sr_no

In [ ]:
srs2 = df[df['subject_no'].isnull() & df['sr_no'].isin(srs)].sr_no
en2 = df[df['subject_no'].isnull() & df['sr_no'].isin(srs)].se_no

In [ ]:
df[df['sr_no'].isin(srs2) | df['se_no'].isin(en2)].sort_values(['sr_no', 'se_no', 'subject_no', 'gender', 'race', 'age', 'birth_year', 'armed', 'injured'])

In [ ]:
# Collect groupings based on multiple columns
cols_of_interest = ['se_no', 'sr_no']
df = df.reset_index(drop=True)
coi = df[cols_of_interest].copy()
coi.loc[coi['se_no'].isnull(), 'se_no'] = (coi.loc[coi['se_no'].isnull()].index + 1) * 10000
coi.loc[coi['sr_no'].isnull(), 'sr_no'] = (coi.loc[coi['sr_no'].isnull()].index + 1) * 10000
gid = 1

while coi.shape[0] > 0:
    se_no  = coi.iloc[0]['se_no']
    se_nos = coi[coi['sr_no'].isin(coi[(coi['se_no']==se_no)]['sr_no'].dropna())]['se_no']
    sr_nos = coi[coi['se_no'].isin(se_nos)]
    inds = list(set(se_nos.index) | set(sr_nos.index))
    df.loc[inds, 'group_id'] = gid
    gid +=1
    coi = coi.drop(inds)

In [ ]:
df['group_id'].nunique()

In [4]:
df2 = union_group(df, 'gid', ['se_no', 'sr_no'])

In [11]:
df2[df2['gid']==36]

,trr_id,gender,race,age,birth_year,armed,injured,alleged_injury,subject_no,sr_no,se_no,trr_date,gid
55,86,MALE,BLACK,40.0,1963,No,No,No,38.0,36.0,37.0,2004-02-17,36
56,87,MALE,BLACK,40.0,1963,No,No,No,38.0,37.0,37.0,2004-02-17,36


In [ ]:
# from itertools import combinations
# import networkx as nx
# from datetime import datetime

# def union_group(df, gid, cols, sep = '__', starting_gid=1):
#     st = datetime.now()
#     all_vals = set()
#     temp_cols = []
#     for col in cols:
#         df.loc[df[col].notnull(),'temp_'+col] =\
#             df.loc[df[col].notnull(), col].map(lambda x: col + sep + str(x))
        
#         assert not all_vals & set(df['temp_'+col].dropna())
#         all_vals.update(set(df['temp_'+col].dropna()))
#         temp_cols.append('temp_'+col)
#     print('here1', datetime.now() - st)
#     el = []
#     for i,r in df[temp_cols].drop_duplicates().iterrows():
#         vals = r.dropna().tolist()
#         if len(vals) > 1:
#             els = list(itertools.combinations(vals,2))
#             el.extend([[i] + list(e) for e in els])
#         else:
#             el.append([i, vals[0], vals[0]])
#     cc = nx.connected_components(nx.from_pandas_dataframe(pd.DataFrame(el, columns=['id', 'H', 'T']),'H','T'))
#     ccl = []
#     cnt = 1
#     print('here2', datetime.now() - st)
#     for i in cc:
#         ccl.extend(list(zip([cnt]*len(i), i)))
#         cnt+=1
#     node_df = pd.DataFrame(ccl, columns=['gid', 'node'])
#     out_df = pd.DataFrame()
#     df.insert(0, 'row_id', df.index)
#     for col in temp_cols:
#         mdf = df[['row_id', col]].merge(node_df, left_on=col, right_on='node', how='inner')
#         print(mdf.head())
#         out_df = out_df.append(mdf[['row_id', 'gid']])
#         df = df.drop(col, axis=1)
#     df = df.merge(out_df.drop_duplicates(), on='row_id', how='left')
#     return df

In [4]:
df2 = union_group(df.copy(), 'gid', ['se_no', 'sr_no'])

In [14]:
gid_T = 125
auid = assign_unique_ids(df2, uid='tid', id_cols=['gid'], conflict_cols=['race', 'birth_year', 'subject_no', 'age', 'gender'])

Total rows: 67019
Unique id + conflict rows: 45439
Conflict rows: 9866
Resolved conflcit UID count: 9144
Total unique ids: 44717



In [16]:
auid[auid.gid==gid_T].sort_values('tid')

,trr_id,gender,race,age,birth_year,armed,injured,alleged_injury,subject_no,sr_no,se_no,trr_date,gid,tid
226,299,MALE,HISPANIC,47.0,1957,No,No,No,136.0,131.0,130.0,2004-03-19,125,35603
228,301,MALE,HISPANIC,30.0,1974,No,No,No,138.0,131.0,130.0,2004-03-19,125,35604
229,302,FEMALE,WHITE,30.0,1974,No,No,No,138.0,131.0,130.0,2004-03-19,125,35605
231,304,MALE,WHITE,26.0,1978,No,No,No,139.0,131.0,130.0,2004-03-19,125,35606
227,300,MALE,WHITE,23.0,1980,No,No,No,137.0,131.0,130.0,2004-03-19,125,35607
230,303,MALE,WHITE,23.0,1980,No,No,No,137.0,131.0,130.0,2004-03-19,125,35607


In [23]:
tdf = pd.DataFrame({
        'A' : [1,1,1,1],
        'B' : [1, np.nan, np.nan, 2],
        'C' : [1, 1, np.nan,2],
        'D' : [1, np.nan, 1, np.nan]
})
resolve_conflicts(df=tdf, id_cols=['A'], conflict_cols=['B','C','D'],uid='tid', start_uid=1)

(   tid  A    B    C    D
 0    2  1  1.0  1.0  1.0
 1    3  1  NaN  1.0  NaN
 2    4  1  NaN  NaN  1.0
 3    5  1  2.0  2.0  NaN, 5)

In [8]:
df2.dropna(subset=['gid']).shape

(67019, 13)

In [10]:
keep_duplicates(df2, 'gid').shape

(45303, 13)

In [ ]:
tt = keep_conflicts(df2[['gid', 'group_id']].drop_duplicates(), all_dups=True)
df2[df2.gid.isin(tt['gid']) | df2.group_id.isin(tt['group_id'])][['gid', 'group_id', 'se_no', 'sr_no']].drop_duplicates().sort_values('gid')

In [ ]:
dfu = df2[['gid', 'gender', 'race', 'age', 'birth_year', 'armed', 'injured', 'alleged_injury', 'subject_no', 'sr_no', 'se_no', 'trr_date']].drop_duplicates()

In [ ]:
keep_duplicates(dfu, 'gid')

In [ ]:
coi = df[cols_of_interest].copy()
coi.loc[coi['se_no'].isnull(), 'se_no'] = (coi.loc[coi['se_no'].isnull()].index + 1) * 10000
coi.loc[coi['sr_no'].isnull(), 'sr_no'] = (coi.loc[coi['sr_no'].isnull()].index + 1) * 10000
coi['sr_str'] = coi['sr_no'].map(lambda x: 'SR-' + str(int(x)))
coi['se_str'] = coi['se_no'].map(lambda x: 'SE-' + str(int(x)))

In [ ]:
import networkx as nx
G=nx.from_pandas_dataframe(coi[['sr_str', 'se_str']], 'sr_str', 'se_str')
list(nx.connected_components(G))

In [ ]:
rd = remove_duplicates(dfu, ['group_id'])
kd = keep_duplicates(dfu, ['group_id'])
dfu[dfu['subject_no'].isin(set(kd['subject_no']) & set(rd['subject_no']))].sort_values('subject_no')

In [ ]:
dfu